In [1]:
# Set local python and nltk paths
import sys
sys.path.insert(0,'/vol/medic02/users/ag6516/miniconda/envs/TheanoLasagne/lib/python2.7/site-packages')
sys.path.insert(0,'/vol/medic02/users/ag6516/miniconda/envs/TheanoLasagne/lib/python2.7/site-packages/setuptools-27.2.0-py2.7.egg')
sys.path.insert(0,'/vol/medic02/users/ag6516/miniconda/envs/TheanoLasagne/lib/python2.7/site-packages/IPython/extensions')
sys.path

['/vol/medic02/users/ag6516/miniconda/envs/TheanoLasagne/lib/python2.7/site-packages/IPython/extensions',
 '/vol/medic02/users/ag6516/miniconda/envs/TheanoLasagne/lib/python2.7/site-packages/setuptools-27.2.0-py2.7.egg',
 '/vol/medic02/users/ag6516/miniconda/envs/TheanoLasagne/lib/python2.7/site-packages',
 '',
 '/vol/medic02/users/ag6516/miniconda/envs/TheanoLasagne/lib/python27.zip',
 '/vol/medic02/users/ag6516/miniconda/envs/TheanoLasagne/lib/python2.7',
 '/vol/medic02/users/ag6516/miniconda/envs/TheanoLasagne/lib/python2.7/plat-linux2',
 '/vol/medic02/users/ag6516/miniconda/envs/TheanoLasagne/lib/python2.7/lib-tk',
 '/vol/medic02/users/ag6516/miniconda/envs/TheanoLasagne/lib/python2.7/lib-old',
 '/vol/medic02/users/ag6516/miniconda/envs/TheanoLasagne/lib/python2.7/lib-dynload',
 '/homes/ag6516/.local/lib/python2.7/site-packages',
 '/vol/medic02/users/ag6516/miniconda/envs/TheanoLasagne/lib/python2.7/site-packages',
 '/vol/medic02/users/ag6516/miniconda/envs/TheanoLasagne/lib/python

In [2]:
%matplotlib inline

import os
import pickle
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import random
from random import randint
random.seed(123)
np.random.seed(123)

# Set THEANO_FLAGS='device=cuda0,floatX=float32' to run notebook on gpu
import lasagne
from lasagne.utils import floatX
import theano
import theano.tensor as T
from collections import Counter
from lasagne.utils import floatX

ERROR (theano.gpuarray): Could not initialize pygpu, support disabled
Traceback (most recent call last):
  File "/vol/medic02/users/ag6516/miniconda/envs/TheanoLasagne/lib/python2.7/site-packages/theano/gpuarray/__init__.py", line 164, in <module>
    use(config.device)
  File "/vol/medic02/users/ag6516/miniconda/envs/TheanoLasagne/lib/python2.7/site-packages/theano/gpuarray/__init__.py", line 151, in use
    init_dev(device)
  File "/vol/medic02/users/ag6516/miniconda/envs/TheanoLasagne/lib/python2.7/site-packages/theano/gpuarray/__init__.py", line 60, in init_dev
    sched=config.gpuarray.sched)
  File "pygpu/gpuarray.pyx", line 634, in pygpu.gpuarray.init
  File "pygpu/gpuarray.pyx", line 584, in pygpu.gpuarray.pygpu_init
  File "pygpu/gpuarray.pyx", line 1057, in pygpu.gpuarray.GpuContext.__cinit__
GpuArrayException: cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


In [3]:
dir = ('/vol/medic02/users/ag6516/x_ray_fracture_localisation/')
# dir = ('/Users/Aydan/PhD/x_ray_fracture_localisation/')

In [4]:
#sample_exams = pickle.load(open('sample_exams_bbox_features.pkl'))
sample_exams = pickle.load(open('augmented_exams_bbox_features.pkl'))
print len(sample_exams)

2154


In [5]:
counter = 0
for item in sample_exams:
    if item['split']!='train':
        #print item

        counter = counter +1
print counter

66


In [6]:
allwords = Counter()
for item in sample_exams:
    tokens = item['tokens']
    #for sentence in item['sentences']:
    allwords.update(tokens)
        
vocab = [k for k, v in allwords.items()]
vocab.insert(0, '#START#')
vocab.append('#END#')

word_to_index = {w: i for i, w in enumerate(vocab)}
index_to_word = {i: w for i, w in enumerate(vocab)}

len(vocab)

167

In [7]:
exam_ids = []
for folder in os.listdir(dir+'data/Images'):
    #os.path.exists(self.labelfilename)
    exam_ids.append(str(folder))

In [8]:
print len(exam_ids)

330


## Max Pool CNN image embeddings

In [ ]:
pids = [item['patient id'] for item in sample_exams]
pids = set(pids)
print len(pids)

In [ ]:
print sample_exams[2]['images'][0]['bbox cnn features']

In [ ]:
pooled_sample_exams = []
for exam in sample_exams:
    item = exam
    all_cnn_features = [image['bbox cnn features'] for image in exam['images']]
    all_cnn_features_array = np.array(all_cnn_features)
    #print all_cnn_features_array.shape
    item['max_bbox_cnn_features'] = np.max(all_cnn_features_array, axis=0)
    pooled_sample_exams.append(exam)

In [ ]:
print pooled_sample_exams[0]['max_bbox_cnn_features'].shape

In [ ]:
pickle.dump(pooled_sample_exams, open('aug_exams_bbox_pooled_features.pkl','w'), protocol=pickle.HIGHEST_PROTOCOL)

# Build Image Caption Model
<img src="caption_model.png">

Implemented in Theano/Lasagne based on __[pydata2015 tutorial](https://github.com/ebenolson/pydata2015/tree/master/4%20-%20Recurrent%20Networks)__ for natural image captioning

In [9]:
pooled_sample_exams = pickle.load(open('aug_exams_bbox_pooled_features.pkl'))
print len(pooled_sample_exams)

2154


In [10]:
SEQUENCE_LENGTH = 33
MAX_SENTENCE_LENGTH = SEQUENCE_LENGTH - 3 # 1 for image, 1 for start token, 1 for end token
#BATCH_SIZE = 50
BATCH_SIZE = 30
CNN_FEATURE_SIZE = 1024
EMBEDDING_SIZE = 256

In [11]:
# Returns a list of tuples (cnn features, list of words, image ID)
def get_data_batch(dataset, size, split='train'):
    items = []
    
    while len(items) < size:
        item = random.choice(dataset)
        if item['split'] != split:
            continue
        sentence = item['tokens']
        if len(sentence) > MAX_SENTENCE_LENGTH:
            sentence = sentence[1:MAX_SENTENCE_LENGTH]
            #continue
        items.append((item['max_bbox_cnn_features'], sentence, item['patient id']))
    
    return items

In [12]:
# Convert a list of tuples into arrays that can be fed into the network
def prep_batch_for_network(batch):
    x_cnn = floatX(np.zeros((len(batch), CNN_FEATURE_SIZE)))
    x_sentence = np.zeros((len(batch), SEQUENCE_LENGTH - 1), dtype='int32')
    y_sentence = np.zeros((len(batch), SEQUENCE_LENGTH), dtype='int32')
    mask = np.zeros((len(batch), SEQUENCE_LENGTH), dtype='bool')

    for j, (cnn_features, sentence, _) in enumerate(batch):
        x_cnn[j] = cnn_features
        i = 0
        for word in ['#START#'] + sentence + ['#END#']:
            if word in word_to_index:
                mask[j, i] = True
                y_sentence[j, i] = word_to_index[word]
                x_sentence[j, i] = word_to_index[word]
                i += 1
        #mask[j, 0] = False
                
    return x_cnn, x_sentence, y_sentence, mask

In [13]:
# sentence embedding maps integer sequence with dim (BATCH_SIZE, SEQUENCE_LENGTH - 1) to 
# (BATCH_SIZE, SEQUENCE_LENGTH-1, EMBEDDING_SIZE)
l_input_sentence = lasagne.layers.InputLayer((BATCH_SIZE, SEQUENCE_LENGTH - 1))
l_sentence_embedding = lasagne.layers.EmbeddingLayer(l_input_sentence,
                                                     input_size=len(vocab),
                                                     output_size=EMBEDDING_SIZE,
                                                    )

# cnn embedding changes the dimensionality of the representation from 1024 to EMBEDDING_SIZE, 
# and reshapes to add the time dimension - final dim (BATCH_SIZE, 1, EMBEDDING_SIZE)
l_input_cnn = lasagne.layers.InputLayer((BATCH_SIZE, CNN_FEATURE_SIZE))
l_cnn_embedding = lasagne.layers.DenseLayer(l_input_cnn, num_units=EMBEDDING_SIZE,
                                            nonlinearity=lasagne.nonlinearities.identity)

l_cnn_embedding = lasagne.layers.ReshapeLayer(l_cnn_embedding, ([0], 1, [1]))

# the two are concatenated to form the RNN input with dim (BATCH_SIZE, SEQUENCE_LENGTH, EMBEDDING_SIZE)
l_rnn_input = lasagne.layers.ConcatLayer([l_cnn_embedding, l_sentence_embedding])


l_dropout_input = lasagne.layers.DropoutLayer(l_rnn_input, p=0.5)

l_lstm = lasagne.layers.LSTMLayer(l_dropout_input,
                                  num_units=EMBEDDING_SIZE,
                                  unroll_scan=True,
                                  grad_clipping=5.)

l_dropout_output = lasagne.layers.DropoutLayer(l_lstm, p=0.5)

# the RNN output is reshaped to combine the batch and time dimensions
# dim (BATCH_SIZE * SEQUENCE_LENGTH, EMBEDDING_SIZE)
l_shp = lasagne.layers.ReshapeLayer(l_dropout_output, (-1, EMBEDDING_SIZE))

# decoder is a fully connected layer with one output unit for each word in the vocabulary
l_decoder = lasagne.layers.DenseLayer(l_shp, num_units=len(vocab), nonlinearity=lasagne.nonlinearities.softmax)

# finally, the separation between batch and time dimension is restored
l_out = lasagne.layers.ReshapeLayer(l_decoder, (BATCH_SIZE, SEQUENCE_LENGTH, len(vocab)))

In [14]:
# cnn feature vector
x_cnn_sym = T.matrix()

# sentence encoded as sequence of integer word tokens
x_sentence_sym = T.imatrix()

# mask defines which elements of the sequence should be predicted
mask_sym = T.imatrix()

# ground truth for the RNN output
y_sentence_sym = T.imatrix()

In [15]:
output = lasagne.layers.get_output(l_out, {
                l_input_sentence: x_sentence_sym,
                l_input_cnn: x_cnn_sym
                })

In [16]:
def calc_cross_ent(net_output, mask, targets):
    # Helper function to calculate the cross entropy error
    preds = T.reshape(net_output, (-1, len(vocab)))
    targets = T.flatten(targets)
    cost = T.nnet.categorical_crossentropy(preds, targets)[T.flatten(mask).nonzero()]
    return cost

loss = T.mean(calc_cross_ent(output, mask_sym, y_sentence_sym))

In [17]:
MAX_GRAD_NORM = 15

all_params = lasagne.layers.get_all_params(l_out, trainable=True)

all_grads = T.grad(loss, all_params)
all_grads = [T.clip(g, -5, 5) for g in all_grads]
all_grads, norm = lasagne.updates.total_norm_constraint(
    all_grads, MAX_GRAD_NORM, return_norm=True)

updates = lasagne.updates.adam(all_grads, all_params, learning_rate=0.00001)

f_train = theano.function([x_cnn_sym, x_sentence_sym, mask_sym, y_sentence_sym],
                          [loss, norm],
                          updates=updates
                         )

f_val = theano.function([x_cnn_sym, x_sentence_sym, mask_sym, y_sentence_sym], loss)

In [18]:
for iteration in range(100000):
    x_cnn, x_sentence, y_sentence, mask = prep_batch_for_network(get_data_batch(pooled_sample_exams, BATCH_SIZE))
    loss_train, norm = f_train(x_cnn, x_sentence, mask, y_sentence)
    if not iteration % 250:
        print('Iteration {}, loss_train: {}, norm: {}'.format(iteration, loss_train, norm))
        try:
            batch = get_data_batch(pooled_sample_exams, BATCH_SIZE, split='val')
            x_cnn, x_sentence, y_sentence, mask = prep_batch_for_network(batch)
            loss_val = f_val(x_cnn, x_sentence, mask, y_sentence)
            print('Val loss: {}'.format(loss_val))
        except IndexError:
            continue

Iteration 0, loss_train: 5.11798620224, norm: 0.47153237462
Val loss: 5.11996221542
Iteration 250, loss_train: 4.74483728409, norm: 0.897225558758
Val loss: 4.79544782639
Iteration 500, loss_train: 4.15996026993, norm: 0.606463491917
Val loss: 4.29827451706
Iteration 750, loss_train: 3.9942214489, norm: 0.546978473663
Val loss: 4.22887182236
Iteration 1000, loss_train: 4.01808023453, norm: 0.43532627821
Val loss: 4.04338502884
Iteration 1250, loss_train: 3.96464633942, norm: 0.464104354382
Val loss: 4.11902952194
Iteration 1500, loss_train: 3.94541192055, norm: 0.396647065878
Val loss: 4.04048442841
Iteration 1750, loss_train: 3.90302729607, norm: 0.508979678154
Val loss: 4.02652406693
Iteration 2000, loss_train: 3.9363117218, norm: 0.45281213522
Val loss: 3.97546458244
Iteration 2250, loss_train: 3.97146129608, norm: 0.503706634045
Val loss: 4.02056312561
Iteration 2500, loss_train: 3.86403107643, norm: 0.682198047638
Val loss: 3.9173989296
Iteration 2750, loss_train: 3.92726635933, n

Iteration 23500, loss_train: 2.50849413872, norm: 1.84056401253
Val loss: 3.18845820427
Iteration 23750, loss_train: 2.59103369713, norm: 1.58087468147
Val loss: 3.1325275898
Iteration 24000, loss_train: 2.45713877678, norm: 1.21170711517
Val loss: 2.99557638168
Iteration 24250, loss_train: 2.5453555584, norm: 2.16495919228
Val loss: 2.92906332016
Iteration 24500, loss_train: 2.49381995201, norm: 1.39315342903
Val loss: 3.15862369537
Iteration 24750, loss_train: 2.37597894669, norm: 1.53953135014
Val loss: 3.26950788498
Iteration 25000, loss_train: 2.32604694366, norm: 1.7332597971
Val loss: 3.29300761223
Iteration 25250, loss_train: 2.48542332649, norm: 1.5904365778
Val loss: 3.17182660103
Iteration 25500, loss_train: 2.40562677383, norm: 1.44222640991
Val loss: 3.006524086
Iteration 25750, loss_train: 2.3853533268, norm: 1.33662474155
Val loss: 3.11103844643
Iteration 26000, loss_train: 2.44840550423, norm: 1.79363083839
Val loss: 3.0428237915
Iteration 26250, loss_train: 2.438640832

Iteration 47000, loss_train: 1.88040304184, norm: 1.99961519241
Val loss: 3.29884696007
Iteration 47250, loss_train: 1.89502048492, norm: 1.84867465496
Val loss: 2.99142122269
Iteration 47500, loss_train: 1.78564620018, norm: 1.55723977089
Val loss: 3.11719512939
Iteration 47750, loss_train: 1.84905159473, norm: 1.54200208187
Val loss: 3.66166162491
Iteration 48000, loss_train: 1.76266753674, norm: 1.55755746365
Val loss: 3.09131932259
Iteration 48250, loss_train: 1.85749363899, norm: 1.68874537945
Val loss: 3.12291121483
Iteration 48500, loss_train: 1.85549390316, norm: 1.72529315948
Val loss: 2.91463446617
Iteration 48750, loss_train: 1.80429935455, norm: 1.69543004036
Val loss: 2.88680028915
Iteration 49000, loss_train: 1.84607338905, norm: 1.64996373653
Val loss: 3.20621705055
Iteration 49250, loss_train: 1.75817012787, norm: 1.92368006706
Val loss: 3.27400517464
Iteration 49500, loss_train: 1.8195387125, norm: 2.41644692421
Val loss: 3.2839910984
Iteration 49750, loss_train: 1.899

Iteration 70500, loss_train: 1.45651066303, norm: 1.83062124252
Val loss: 3.47296118736
Iteration 70750, loss_train: 1.47821485996, norm: 1.66563725471
Val loss: 3.29878544807
Iteration 71000, loss_train: 1.51640331745, norm: 1.66282558441
Val loss: 3.42839336395
Iteration 71250, loss_train: 1.39008867741, norm: 2.28835844994
Val loss: 3.22186660767
Iteration 71500, loss_train: 1.38287436962, norm: 1.7904446125
Val loss: 3.57697224617
Iteration 71750, loss_train: 1.40616297722, norm: 2.35901856422
Val loss: 3.33194541931
Iteration 72000, loss_train: 1.43934226036, norm: 2.12216162682
Val loss: 3.6637468338
Iteration 72250, loss_train: 1.32425796986, norm: 1.55065560341
Val loss: 3.2310693264
Iteration 72500, loss_train: 1.35271716118, norm: 1.53700697422
Val loss: 3.4615175724
Iteration 72750, loss_train: 1.3802267313, norm: 1.63140928745
Val loss: 3.58521080017
Iteration 73000, loss_train: 1.38844239712, norm: 1.72284531593
Val loss: 3.48099303246
Iteration 73250, loss_train: 1.389422

Iteration 94000, loss_train: 1.18915867805, norm: 1.62773573399
Val loss: 3.86365413666
Iteration 94250, loss_train: 1.17069578171, norm: 1.65645492077
Val loss: 3.33945560455
Iteration 94500, loss_train: 1.14403951168, norm: 1.79713153839
Val loss: 3.81969618797
Iteration 94750, loss_train: 1.19704449177, norm: 2.00193953514
Val loss: 3.99678134918
Iteration 95000, loss_train: 1.10642015934, norm: 2.39348077774
Val loss: 3.86284184456
Iteration 95250, loss_train: 1.12835133076, norm: 1.73801875114
Val loss: 4.09640645981
Iteration 95500, loss_train: 1.12133646011, norm: 1.72093307972
Val loss: 3.87421703339
Iteration 95750, loss_train: 1.11028277874, norm: 1.91868042946
Val loss: 3.49869346619
Iteration 96000, loss_train: 1.2336666584, norm: 1.6736741066
Val loss: 3.41858124733
Iteration 96250, loss_train: 1.12607419491, norm: 2.03186178207
Val loss: 4.05981683731
Iteration 96500, loss_train: 1.18508386612, norm: 1.86972308159
Val loss: 3.65179586411
Iteration 96750, loss_train: 1.100

In [19]:
param_values = lasagne.layers.get_all_param_values(l_out)
d = {'param values': param_values,
     'vocab': vocab,
     'word_to_index': word_to_index,
     'index_to_word': index_to_word,
    }
pickle.dump(d, open('aug_maxpooling_bbox_trained_v_167.pkl','w'), protocol=pickle.HIGHEST_PROTOCOL)